In [1]:
import prehook

df_list = prehook.populate_dfs()

for df in df_list:
    table_name = df.get('table_name')
    file_id = df.get('file_id')
    message = f"Table name = {table_name} and file_id = {file_id}"
    print(message)


AttributeError: module 'prehook' has no attribute 'populate_dfs'

In [9]:
import prehook
import lookups
import database_handler

db_session = database_handler.create_connection()
prehook.execute(db_session)
database_handler.close_connection(db_session)


column "id" specified more than once

